# 1. User Account Data Scraper

In [ ]:
# How to find and extract the encryption rule hidden in the JavaScript of the webpage:
# JS → index_10ae3b3.js → _signature → _bytedAcrawler → 
# douyin_falcon:node_modules/byted-acrawler/dist/runtime → third_54fd252.js → 

import os
import re
import sys
import json
import time
import random
#import asyncio
#import aiohttp
import datetime
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import urllib.request
from time import sleep

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
video_headers ={'authority': 'aweme.snssdk.com','method': 'GET','scheme': 'https',
'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36',
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,\
image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-US,en;q=0.9',
'upgrade-insecure-requests': '1',
'cache-control': 'max-age=0',
'sec-fetch-mode': 'navigate',
'sec-fetch-site': 'none',
'sec-fetch-user': '?1'}

def proxy():
    proxy = requests.get('http://api.ip.data5u.com/dynamic/\
    get.html?order=8ed0b4d65dffaa785c0ccc76b5929c59&sep=3').text.strip()
    print('the current ip: '+proxy.split(':')[0])
    return {'http':proxy,'https':proxy}
def ip():
    re = requests.get('http://www.882667.com/',proxies = proxy())
    soup = bs.BeautifulSoup(re.content,'html.parser')
    return 'current ip: '+[i.text for i in soup.find_all('p')\
                           if '你的ip是' in str(i)][0].split('\u3000')[0].split('：')[1]
def generate_path(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
def video(video_url,file_name):
    return urllib.request.urlretrieve(requests.get(video_url,headers = video_headers).url,file_name)
def cover(cover_url,file_name):
    return urllib.request.urlretrieve(cover_url,file_name)
def real_time(timestamp):
    return str(datetime.datetime.fromtimestamp(timestamp))
def param(share_url):
    re = requests.get(share_url,headers=headers)
    #uid=re.text.split('uid: ')[1][1:12]
    dytk=re.text.split('dytk: ')[1][1:33]
    nickname=re.text.split('nickname">')[1].split('</p>')[0]
    if '&u_code=' in re.url:
        sec_uid=re.url.split('sec_uid=')[1].split('&u_code=')[0]
    else:
        sec_uid=re.url.split('sec_uid=')[1].split('&utm_')[0] 
    return [sec_uid,dytk,nickname]
def get_api_(sec_uid,max_cursor,dytk):
    API=[]
    def page(sec_uid,max_cursor,dytk):
        url='https://www.amemv.com/web/api/v2/aweme/post/?'
        params={'sec_uid':sec_uid,'count':'21','max_cursor':max_cursor,'aid':'1128','dytk':dytk}
        re=requests.get(url,headers = headers, params=params)
        #print(re.url,'\n')
        API.append(re.url)
        data = re.json()
        if data['has_more'] == True:
            return page(sec_uid, data['max_cursor'], dytk)
            time.sleep(random.uniform(6,8))
        else:
            return API
    return page(sec_uid,max_cursor,dytk)
def get_api(share_url):
    p=param(share_url)
    get_api_(p[0],'0',p[1])
    return get_api_(p[0],'0',p[1])
def douyin_account(share_url):
    API=get_api(share_url)
    num_api = 0
    for api in API:
        print(f'API {num_api},'+' '+api,'\n')
        num_api=num_api+1
        re = requests.get(api).json()
        data = re['aweme_list']
        desc = [info['desc'] for info in data]
        nickname = [info['author']['nickname'] for info in data]
        aweme_id = [info['statistics']['aweme_id'] for info in data]
        share_count = [info['statistics']['share_count'] for info in data]
        forward_count = [info['statistics']['forward_count'] for info in data]
        like_count = [info['statistics']['digg_count'] for info in data]
        comment_count = [info['statistics']['comment_count'] for info in data]
        cover_url = [info['video']['cover']['url_list'][0] for info in data]
        cover_visual = ['<img src="'+ url + '" width="100" >' for url in cover_url]
        aweme_id = [str(info['statistics']['aweme_id'])+'\t' for info in data]

        hashtag_name=[]
        for info in re['aweme_list']:
            hashtag_name_=[]
            for info_ in info['text_extra']:
                try:
                    h=str(info_['hashtag_name'])
                except:
                    h=str('')
                if len(h)>0 and h!='0':
                    hashtag_name_.append(h)
            hashtag_name.append(', '.join(hashtag_name_))
    
        hashtag_id=[]
        for info in re['aweme_list']:
            hashtag_id_=[]
            for info_ in info['text_extra']:
                try:
                    h=str(info_['hashtag_id'])
                except:
                    h=str('')
                if len(h)>0 and h!='0':
                    hashtag_id_.append(h+'\t')
            hashtag_id.append(', '.join(hashtag_id_))
            
        height = [info['video']['height'] for info in data]
        width = [info['video']['width'] for info in data]
        ratio = [info['video']['ratio'] for info in data]
        duration = [round(info['video']['duration']/1000) for info in data]
        video_url = []
        for info in data:
            video_url.append([i for i in info['video']['download_addr']['url_list'] if 'aweme' in i][0].\
            replace('watermark=1','watermark=0'))
        df=pd.DataFrame({'desc':desc,'nickname':nickname,'share_count':share_count,'forward_count':forward_count,
                         'like_count':like_count,'comment_count':comment_count,'video_url':video_url,
                         'cover_visual':cover_visual, 'video_id': aweme_id,'ratio':ratio,'duration':duration,
                         'hashtag_name':hashtag_name,'hashtag_id':hashtag_id,'height':height,'width':width})
        df['period_from'] = real_time(int(str(re['max_cursor'])[:-3]))
        df['period_to'] = real_time(int(str(re['min_cursor'])[:-3]))
        columns = ['nickname','desc','hashtag_name','hashtag_id','video_id','share_count','forward_count',
                   'like_count','comment_count','video_url','cover_visual','height','width','ratio',
                   'duration','period_from','period_to']
        
        file = os.path.exists('./videos_by_account.csv')
        if not file:
            df.to_csv('./videos_by_account.csv',encoding = 'utf-8-sig', index = False, columns = columns,
                     float_format='%f')
        else:
            df.to_csv('./videos_by_account.csv',encoding = 'utf-8-sig', index = False, columns = columns,
                     float_format='%f', mode='a', header = False)
            
        # video downloading
        generate_path('./videos_by_account')
        for num in range(0,len(df)):
            try:
                video(df['video_url'][num],'./videos_by_account/'+
                      df['nickname'][num]+'-'+df['video_id'][num].replace('\t','')+'.mp4')
                print('Account: '+df['nickname'][num]+', Video: '+df['video_id'][num]+'......Succeeded')
            except:
                print('Account: '+df['nickname'][num]+', Video: '+df['video_id'][num]+'......Failed')
            time.sleep(random.uniform(4,5))
        print('\n')

url_csv = pd.read_excel('ball.xlsx',encoding = 'utf-8-sig')
task_num = 0
for share_url in url_csv['Link'][6:]:
    print('####################################################################')
    print(f'Task: {task_num}, '+'Account: '+param(share_url)[2]+', Share Url: '+share_url,'\n')
    douyin_account(share_url)
    task_num = task_num + 1

# 2. Hot Trending Data Scraper

In [ ]:
import os
import json
import datetime
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import urllib.request
from IPython.core.display import HTML
import warnings; warnings.filterwarnings('ignore')

video_headers ={'authority': 'aweme.snssdk.com','method': 'GET','scheme': 'https',
'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36',
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,\
image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-US,en;q=0.9',
'upgrade-insecure-requests': '1',
'cache-control': 'max-age=0',
'sec-fetch-mode': 'navigate',
'sec-fetch-site': 'none',
'sec-fetch-user': '?1'}

def proxy():
    proxy = requests.get('http://api.ip.data5u.com/dynamic/get.html?order=8ed0b4d65dffaa785c0ccc76b5929c59&sep=3').text.strip()
    print('the current ip: '+proxy.split(':')[0])
    return {'http':proxy,'https':proxy}
def ip():
    re = requests.get('http://www.882667.com/',proxies = proxy())
    soup = bs.BeautifulSoup(re.content,'html.parser')
    return 'current ip: '+[i.text for i in soup.find_all('p') if '你的ip是' in str(i)][0].split('\u3000')[0].split('：')[1]
def generate_path(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
def video(video_url,file_name):
    return urllib.request.urlretrieve(requests.get(video_url,headers=video_headers).url,file_name)
def cover(cover_url,file_name):
    return urllib.request.urlretrieve(cover_url,file_name)
def real_time(timestamp):
    return str(datetime.datetime.fromtimestamp(timestamp))

api = 'https://aweme-hl.snssdk.com/aweme/v1/hot/search/list/?detail_list=1&mac_address=\
08:00:27:29:D2:F5&os_api=23&device_type=MI%205s&device_platform=android&ssmix=a&iid=\
92152480453&manifest_version_code=860&dpi=320&uuid=008796750074613&version_code=\
860&app_name=aweme&version_name=8.6.0&ts=1577932778&openudid=c055533a0591b2dc&device_id=\
69918538596&resolution=810*1440&os_version=6.0.1&language=zh&device_brand=Xiaomi&app_type=\
normal&ac=wifi&update_version_code=8602&aid=1128&channel=tengxun_new&_rticket=1577932779592'
re=requests.get(api)
#print('the trend_ip: '+trend_proxy.split(':')[0])
last_update = re.json()['data']['active_time']
trending_data = re.json()['data']['word_list']
trend = pd.DataFrame(trending_data)
trend['group_id']=trend['group_id'].astype(str)
Word_cover=[]
for i in trend['word_cover']:
    if type(i)==dict:
        Word_cover.append(i['url_list'][0])
    else:
        Word_cover.append(None)

date=last_update.split(' ')[0]
time=last_update.split(' ')[1]
trend = trend.drop(columns = ['word_cover','challenge_id'])
trend['collection_date'] = date
trend['collection_time'] = time
if 'ad_data' in trend.columns:
    trend = trend.drop(columns = ['ad_data','search_word'])
file = os.path.exists('./trend.csv')
columns = ['position','word','video_count','hot_value','collection_date',
           'collection_time','word_type','label','group_id']
if not file:
    trend.to_csv('./trend.csv',encoding = 'utf-8-sig', index = False, columns = columns)
else:
    trend.to_csv('./trend.csv',encoding = 'utf-8-sig', index = False, mode='a', columns = columns, header=False)

# core function
def scraper(topic):
    # data collecting
    topic_api='https://aweme-hl.snssdk.com/aweme/v1/hot/search/video/list/?hotword='
    re=requests.get(topic_api+topic+'&count=2000').json()
    print(topic_api+topic+'&count=2000')
    data = re['aweme_list']
    desc = [info['desc'] for info in data]
    time_stamp = [info['create_time'] for info in data]
    create_time = [real_time(info['create_time']) for info in data]
    nickname = [info['author']['nickname'] for info in data]
    verify = [info['author']['custom_verify'] for info in data]
    share_count = [info['statistics']['share_count'] for info in data]
    forward_count = [info['statistics']['forward_count'] for info in data]
    like_count = [info['statistics']['digg_count'] for info in data]
    comment_count = [info['statistics']['comment_count'] for info in data]
    download_count = [info['statistics']['download_count'] for info in data]
    cover_url = [info['video']['cover']['url_list'][0] for info in data]
    cover_visual = ['<img src="'+ url + '" width="100" >' for url in cover_url]
    aweme_id = [info['statistics']['aweme_id']+'\t' for info in data]
    enterprise_verify_reason = [info['author']['enterprise_verify_reason'] for info in data]
    height = [info['video']['height'] for info in data]
    width = [info['video']['width'] for info in data]
    ratio = [info['video']['ratio'] for info in data]
    duration = [round(info['video']['duration']/1000) for info in data]  

    hashtag_name=[]
    for info in re['aweme_list']:
        hashtag_name_=[]
        for info_ in info['text_extra']:
            try:
                h=str(info_['hashtag_name'])
            except:
                h=str('')
            if len(h)>0 and h!='0':
                hashtag_name_.append(h)
        hashtag_name.append(', '.join(hashtag_name_))
    
    hashtag_id=[]
    for info in re['aweme_list']:
        hashtag_id_=[]
        for info_ in info['text_extra']:
            try:
                h=str(info_['hashtag_id'])
            except:
                h=str('')
            if len(h)>0 and h!='0':
                hashtag_id_.append(h+'\t')
        hashtag_id.append(', '.join(hashtag_id_))
            
          
    video_url = []
    for info in data:
        video_url.append([i for i in info['video']['download_addr']['url_list'] if 'aweme-hl' in i][0].\
                         replace('watermark=1','watermark=0'))
        
    df=pd.DataFrame({'topic_name':topic, 'desc':desc,'hashtag_name':hashtag_name,'hashtag_id':hashtag_id,
                     'nickname':nickname,'create_time':create_time,'verify':verify,'time_stamp':time_stamp,
                     'share_count':share_count,'forward_count':forward_count,'like_count':like_count,
                     'comment_count':comment_count,'download_count':download_count,'video_url':video_url,
                     'cover_visual':cover_visual,'video_id': aweme_id,'height':height,'width':width,
                     'ratio':ratio,'duration':duration,'enterprise_verify_reason':enterprise_verify_reason})
    
    
    columns = ['topic_name','nickname','desc','hashtag_name','hashtag_id','video_id','create_time','duration',
               'share_count','forward_count','like_count','comment_count','download_count','verify',
               'enterprise_verify_reason','time_stamp','video_url','cover_visual','height','width','ratio']
    
    file = os.path.exists('./videos.csv')
    if not file:
        df.to_csv('./videos.csv',encoding = 'utf-8-sig', index = False, columns = columns,float_format='{:f}'.format)
    else:
        df.to_csv('./videos.csv',encoding = 'utf-8-sig', index = False, mode='a',
                                 header = False, columns = columns,float_format='{:f}'.format)
        
    # video downloading
    generate_path('./videos')
    index=0
    for num in range(0,len(data)):
        try:
            video(df['video_url'][num],'./videos/'+topic+'-'+df['video_id'][num].replace('\t','')+'.mp4')
            print(topic+', video #'+str(df['video_id'][num])+'......Successed')
        except:
            print(topic+', video #'+str(df['video_id'][num])+'......Failed')
            continue
def douyin_trend():
    for topic_ in trend['word']:
        scraper(topic_)